<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/fresnet_serret.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import jax.numpy as jnp
import jax
import plotly.figure_factory as ff
import plotly.graph_objects as go
import numpy as np

In [10]:
R=1.
def xy(t):
    return jnp.array([R*jnp.cos(t), R*jnp.sin(t)])

def normxy(t):
    return jnp.linalg.norm(xy(t))

In [26]:
dxy_dt = jax.jacobian(xy)
dxy_norm_dt = jax.grad(normxy)
def tangent(t):
    tang=dxy_dt(t)
    return tang/jnp.linalg.norm(tang)

In [39]:
def dsdt(t):
    return jnp.linalg.norm(dxy_dt(t))

In [40]:
dtangent_dt=jax.jacobian(tangent)
def normal(t):
    return dtangent_dt(t)/dsdt(t)

In [41]:
xy_vec=jnp.vectorize(xy,signature='()->(2)')
tangent_vec=jnp.vectorize(dxy_dt,signature='()->(2)')
normal_vec=jnp.vectorize(normal,signature='()->(2)')

In [42]:

theta=jnp.linspace(0,2*np.pi,30)

x,y = xy_vec(theta).T
T_x,T_y = tangent_vec(theta).T
N_x,N_y=normal_vec(theta).T


In [43]:
fig = ff.create_quiver(x=x, y=y, u=T_x, v=T_y, line_color='blue')
fig2 = ff.create_quiver(x=x,y=y,u=N_x,v=N_y, line_color='red')
fig.add_traces(data=fig2.data)
fig.update_layout(width=600,height=600,template='plotly_dark')

In [44]:
def fresnet_serret(f):
    df_dt = jax.jacobian(f)
    def tangent(t):
        tang=df_dt(t)
        return tang/jnp.linalg.norm(tang)

    def dsdt(t):
        return jnp.linalg.norm(df_dt(t))

    dtangent_dt=jax.jacobian(tangent)
    def normal(t):
        normal=dtangent_dt(t)
        return normal/dsdt(t)

    return tangent, normal

In [45]:
#viviani's curve
R=1.
def xyz(t):
    x=R*jnp.cos(t)*jnp.cos(t)
    y=R*jnp.cos(t)*jnp.sin(t)
    z=R*jnp.sin(t)
    return jnp.array([x,y,z])

In [46]:
T, N = fresnet_serret(xyz)

In [47]:
xyz_vec = jnp.vectorize(xyz, signature='()->(3)')
T_vec = jnp.vectorize(T, signature='()->(3)')
N_vec = jnp.vectorize(N, signature='()->(3)')

In [48]:
theta=jnp.linspace(0,2*np.pi,30)

x,y,z = xyz_vec(theta).T
T_x,T_y,T_z = T_vec(theta).T
N_x,N_y,N_z=N_vec(theta).T



In [50]:
fig = go.Figure()
color_tangent=[
        [0, "rgb(200, 0, 0)"],
        [1., "rgb(200, 0, 0)"]]
color_normal=[
        [0, "rgb(0, 200, 0)"],
        [1., "rgb(0, 200, 0)"]]
fig.add_trace(go.Cone(x=x,y=y,z=z, u=T_x, v=T_y, w=T_z, anchor='tail', colorscale=color_tangent))
fig.add_trace(go.Cone(x=x,y=y,z=z, u=N_x, v=N_y, w=N_z,anchor='tail', colorscale=color_normal))
fig.update_traces(showscale=False, sizeref=1,)
fig.update_layout(width=600, height=600, template='plotly_dark')